# 선형회귀(Linear Regression)

선형회귀 개념에 대한 정리는 제 Notion에 정리가 되어 있습니다.

https://noversezero.notion.site/9884bad5e8d14d4aabaf38dad2dd2e25

In [1]:
#기본세팅

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
x_train = torch.FloatTensor([[1], [2], [3]])
y_train = torch.FloatTensor([[2], [4], [6]])

print(x_train)
print(x_train.shape)

print(y_train)
print(y_train.shape)

tensor([[1.],
        [2.],
        [3.]])
torch.Size([3, 1])
tensor([[2.],
        [4.],
        [6.]])
torch.Size([3, 1])


우리는 Y = Wx + B로 되어 있는 Linear Regression을 진행할 것이다.

우리의 최종 목표는 비용함수(Cost Function)이 최소화되는 직선을 찾는 것이다.

그러기 위해서는 초기 Weight 값과 Bias 값을 설정해야하기 때문에 0으로 초기화 시킬 것이다.

In [3]:
# 가중치 W와 편향 b를 0으로 초기화하고 학습을 통해 값이 변경되는 변수임을 명시함.
W = torch.zeros(1, requires_grad=True) 
b = torch.zeros(1, requires_grad=True)

# 가중치 W를 출력
print(W) 

# 편향 b를 출력
print(b)

tensor([0.], requires_grad=True)
tensor([0.], requires_grad=True)


현재 우리의 방정식은 Y = 0x + 0 이다.

## 가설  세우기

우리의 가설은 가중치 하나로 이루어져 있는 Linear Regression Model이다.

In [4]:
hypothesis = x_train * W + b
print(hypothesis)

tensor([[0.],
        [0.],
        [0.]], grad_fn=<AddBackward0>)


## 비용함수 선언하기

우리의 비용함수는 MSE(Mean Squared Error)이다.

In [5]:
# 앞서 배운 torch.mean으로 평균을 구한다.
cost = torch.mean((hypothesis - y_train) ** 2) 
print(cost)

tensor(18.6667, grad_fn=<MeanBackward0>)


## Optimizer 설정

확률적 경사하강법(SGD, Stochastic Gradient Descent)은 매 스텝(step)에서 딱 1개의 샘플을 무작위로 선택하고 그에 대한 gradient를 계산하는 것이다.

매 스텝에서 1개의 샘플을 무작위로 선정하기 때문에 샘플 전체를 사용하는 것보다 더 빠르게 진행 될 수 있다.

따라서 매우 큰 데이터에 적용하기 좋다.

이와 반대로 모든 샘플을 이용하는 경사하강법은 배치 경사하강법이다.

위에 링크를 걸어 놓은 내 Notion에 있는 gradient 계산은 배치 경사하강법을 위한 gradient 계산이라고 할 수 있다.

In [8]:
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.01) #이때 lr은 Learning rate이다.

우리가 Gradient descent를 구현하기 위해서는 3가지 함수를 사용할 것이다.

optimizer.zero_grad() : gradient를 0으로 초기화

cost.backward() : 비용 함수를 미분하여 gradient 계산

optimizer.step() : W와 b를 업데이트

gradient 초기화 -> gradient 계산 -> W와 b Update를 정해진 횟수만큼 반복할 것이다.

이때 왜 gradient를 초기화해야하냐면, Pytorch는 Gradient를 계속 누적시키는 특징이 있다.

그렇기 때문에 gradient를 0으로 초기화하지 않으면, 새로 구한 gradient와 이전의 gradient의 합으로 W와 b를 Update하게 된다.

따라서 gradient를 초기화해야지 다음 gradient로 Update가 가능하다.

In [9]:
nb_epochs = 1999 # 원하는만큼 경사 하강법을 반복
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x_train * W + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, W.item(), b.item(), cost.item()
        ))

Epoch    0/1999 W: 0.187, b: 0.080 Cost: 18.666666
Epoch  100/1999 W: 1.746, b: 0.578 Cost: 0.048171
Epoch  200/1999 W: 1.800, b: 0.454 Cost: 0.029767
Epoch  300/1999 W: 1.843, b: 0.357 Cost: 0.018394
Epoch  400/1999 W: 1.876, b: 0.281 Cost: 0.011366
Epoch  500/1999 W: 1.903, b: 0.221 Cost: 0.007024
Epoch  600/1999 W: 1.924, b: 0.174 Cost: 0.004340
Epoch  700/1999 W: 1.940, b: 0.136 Cost: 0.002682
Epoch  800/1999 W: 1.953, b: 0.107 Cost: 0.001657
Epoch  900/1999 W: 1.963, b: 0.084 Cost: 0.001024
Epoch 1000/1999 W: 1.971, b: 0.066 Cost: 0.000633
Epoch 1100/1999 W: 1.977, b: 0.052 Cost: 0.000391
Epoch 1200/1999 W: 1.982, b: 0.041 Cost: 0.000242
Epoch 1300/1999 W: 1.986, b: 0.032 Cost: 0.000149
Epoch 1400/1999 W: 1.989, b: 0.025 Cost: 0.000092
Epoch 1500/1999 W: 1.991, b: 0.020 Cost: 0.000057
Epoch 1600/1999 W: 1.993, b: 0.016 Cost: 0.000035
Epoch 1700/1999 W: 1.995, b: 0.012 Cost: 0.000022
Epoch 1800/1999 W: 1.996, b: 0.010 Cost: 0.000013
Epoch 1900/1999 W: 1.997, b: 0.008 Cost: 0.000008

위의 출력된 결과를 보면 Cost가 계속 줄어드는 것을 볼 수 있다.

처음에는 18.666이 였지만 1900 반복했을때는 0.000008까지 줄어드는 것을 확인할 수 있다.

## 자동 미분(Autograd)

PyTorch는 우리가 만드는 Tensor에 requires_grad = True를 입력하면 자동으로 gradient를 계산해준다.

모델이 복잡해질수록 우리가 Gradient descent를 직접 코딩하는 것은 매우 복잡한 일이 될 것이다.

하지만 PyTorch의 자동 미분 기능이 있기 때문에 우리는 backward()함수를 통해 미분을 쉽게 할 수 있다.

In [11]:
w = torch.tensor(2.0, requires_grad=True) #requires_grad 옵션을 통해 자동미분 기능을 갖고 있는 tensor를 생성하였다.

y = w**2
z = 2*y + 5 
# w의 제곱에 2를 곱하고 5를 더하는 방정식을 생성하였다. 

#이때 우리가 직접 미분하는 코드를 코딩할 필요가 없고 backward() 함수를 사용하면 손쉽게 미분이 가능하다.
z.backward()
# 이제 미분을 한 결과를 grad() 함수를 통해 출력하면, 미분한 값이 나온다.
print('수식을 w로 미분한 값 : {}'.format(w.grad))

수식을 w로 미분한 값 : 8.0
